<a href="https://colab.research.google.com/github/ChangQi-555/ielts/blob/master/Sep_1_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 爬蟲!

In [1]:
!pip install selenium
!pip install fake_useragent
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common import action_chains as ActionChains
from selenium.webdriver.chrome.options import Options
opt = Options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--incognito')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 968 kB 9.4 MB/s 
     |████████████████████████████████| 359 kB 58.8 MB/s 
     |████████████████████████████████| 138 kB 68.7 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 3.6 MB 46.3 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=7b567ac52e63

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
import json
from google.colab import drive
import datetime
drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


##  測試window handles 切換頁面

In [26]:
from fake_useragent import UserAgent 
ua = UserAgent()
user_agent = ua.random
chrome_options.add_argument('user-agent={}'.format(ua))  
wd.get("https://www.tripadvisor.com.tw/")
wait = WebDriverWait(wd, 20)


search_button = wait.until(ec.element_to_be_clickable((By.XPATH, "//*[@id='lithium-root']/main/div[3]/div/div/div[2]/form/input[1]")))  
search_button.click()

search_button.send_keys("涵碧樓")
search_button.send_keys(Keys.ENTER)
parent_handle = wd.current_window_handle
print('1: ' + wd.title)
ele = wait.until(ec.element_to_be_clickable((By.XPATH,"//*[@id='BODY_BLOCK_JQUERY_REFLOW']/div[2]/div/div[2]/div/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[1]/div/div[2]/div/div/div[1]/div/div/div/div[2]/div[1]/div[1]/span")))
ele.click()
handles = wd.window_handles
size = len(handles)
# for x in range(size):
#   print('x= ' + str(x) )
#   wd.switch_to.window(handles[x])
#   print('child window: ' + wd.title)
wd.switch_to.window(handles[1])
print('child window: ' + wd.title)
time.sleep(7)
# print('2: ' + wd.title)
heading = wait.until(ec.presence_of_element_located((By.XPATH,"//*[@id='HEADING']")))
print(heading.text)


1: 搜尋結果： 涵碧樓 - Tripadvisor
child window: 日月潭涵碧樓酒店 (魚池) - 250 則旅客評論和比價
日月潭涵碧樓酒店 (魚池)
The Lalu Sun Moon Lake


## switch comment page test

In [54]:
wd.get("https://www.tripadvisor.com/Restaurant_Review-g308272-d2661749-Reviews-Jin_Xuan_Chinese_Restaurant-Shanghai.html")
for i in range(1,5):
  titles = wd.find_elements(By.XPATH, "//div//p[@class='partial_entry']")
  #more = wait.until(ec.element_to_be_clickable((By.XPATH, "//div//p//span[@class='taLnk ulBlueLinks']")))
  #more.click()
  for j  in titles:
    print(j.text);
  # print(titles[0].text)
  # print(titles[1].text)
  # print(titles[2].text)
  # print(titles[3].text)
  next_page = wait.until(ec.element_to_be_clickable((By.XPATH, "//div//a[@class = 'pageNum '][" + str(i) + "]")))
  next_page.click()
  
  page_count = page_count + 1
  time.sleep(5)



We had a lovely experience at this restaurant. We made a reservation and we were given a nice table with the perfect view of the Huangpu River. The food was great and the service was impecable (special mention to Janice and Johnson). We also enjoyed...More
Do you need to have a personalized touch? Do you love excellent service AND delicious food? Are you looking for Vegan, Vegetarian friendly dinner set menus? Jin Xuan went above and beyond to give me a memorable, magical Christmas Eve Dinner!
Dear Maliza4Cambodia Thank you for sharing a wonderful review with us. I am delighted to learn that you enjoy dining with us. Our Ladies and Gentlemen look forward to welcoming you back in the near future. Warmest regards Debbie Wong Quality Manager
First off anything associated with the Ritz-Carlton is always five stars, and this restaurant not any different. I entrusted Bill, the sommelier, with my all the food decisions. The menu had so many options, so I ordered all his recommendations. I had

## 正式爬蟲

In [ ]:

wait = WebDriverWait(wd, 20)

page_count = 1
#利用user_agent 去躲避反爬蟲
# dictionary -> list 
dic = {"title_keys" : [],
    "comment_keys" : [],
    "rating_keys" : [],
    "comment_time": []}


comments = []
ratings = []
titles = []
def webcrawling():
  #parameter
  

  
  comment_times_htmls = []
  comment_times = []
  # title
  titles = wd.find_elements(By.XPATH, "//div[@class='fpMxB MC _S b S6 H5 _a']//span//span")

  for title in titles:
    dic['title_keys'].append(title.text)
  time.sleep(10)
  # comment
  comments = wd.find_elements(By.XPATH, "//q[@class='XllAv H4 _a']//span")
  for comment in comments:
    dic['comment_keys'].append(comment.text)

  # rating
  ratings = wd.find_elements(By.XPATH, "//div[@class = 'emWez F1']//span")
  # 判斷是幾星評論
  for rating in ratings: 
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_50"):
      dic['rating_keys'].append("5 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_40"):
      dic['rating_keys'].append("4 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_30"):
      dic['rating_keys'].append("3 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_20"):
      dic['rating_keys'].append("2 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_10"):
      dic['rating_keys'].append("1 STARS")

  #comment time
  comment_times_htmls = wd.find_elements(By.XPATH,"//div[@class = 'bcaHz']//span")
  for comment_time_html in comment_times_htmls:
    if comment_time_html.text[-8:-7] == str(2):
      dic['comment_time'].append(comment_time_html.text[-8:])
    else:
      dic['comment_time'].append(comment_time_html.text[-7:])

#變成比較像 json 的 dictionary form
def jsoncrawling():
  count = 0
  titles_new = []
  comments_new = []
  rating_star = []
  prsn = {} #dictionary
  for title in titles:
    titles_new.append(title.text)
    count += 1
  for comment in comments:
    comments_new.append(comment.text)
  for rating in ratings:
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_50"):
      rating_star.append("5 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_40"):
      rating_star.append("4 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_30"):
      rating_star.append("3 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_20"):
      rating_star.append("2 STARS")
    if(rating.get_attribute("class") == "ui_bubble_rating bubble_10"):
      rating_star.append("1 STARS")
  for i in range(0, count):
    prsn["person " + str(i + 1)] = {"title_key" : '',  "comment_key" : '',   "rating_key" : ''}
  for i in range(1, count+1):
    prsn['person '+str(i)]['title_key'] = titles_new[i-1]
    prsn['person '+str(i)]['comment_key'] = comments_new[i-1]
    prsn['person '+str(i)]['rating_key'] = rating_star[i-1]


for p in range(1,5):
  webcrawling()
  jsoncrawling()
  
  print("p: " + str(p))
  next_page = wait.until(ec.element_to_be_clickable((By.XPATH, "//div//a[@class = 'pageNum '][" + str(p) + "]")))
  next_page.click()
  page_count = page_count + 1
  time.sleep(5)



# for i in range(0 , )
jsdic = json.dumps(prsn, default=lambda o: '<not serializable>',ensure_ascii=False,indent=4, sort_keys=True)
# jsdic = json.dumps(prsn)  #轉成json並寫入 google drive
with open('/content/drive/My Drive/Colab Notebooks/TripadvisorCrawling.json', 'w', encoding='utf-8') as f:
  f.write(jsdic)
  f.close
drive.flush_and_unmount()


print("////////////////////////////////////////////////////")
time.sleep(10)
wd.quit()

p: 1
p: 2
p: 3
p: 4


NameError: ignored

In [ ]:
print(page_count)

2


## 印出dic

In [ ]:
for i in dic:
  print(i)

title_keys
comment_keys
rating_keys
comment_time


In [ ]:
for i in range(0,len(dic['title_keys'])):
  print(dic.get("title_keys")[i])

值得一去的飯店
美好回憶的旅行
春節假期前涵碧樓
極簡、禪風、Ongoing Style。
適合約會，不宜親子
曾經風光的雅致渡假飯店
風景無敵東方餐廳菜色普通用餐環境壅擠
最美的風景是房間內的蜘蛛絲
訂出高房價，就應該提供住客高品質的住宿經驗！涵碧樓該往前走了 !
旅館管理需要加強，旅客隱私沒有保障
古典飯店
環境優美
高質感的住宿環境
美好的地方
地理位置和服務極佳的飯店、有很棒的三溫暖澡堂、飲食水平差強人意
悠閒的地方
最喜歡的飯店
完美的家庭休閒
20190402～0403 第二次住
超級棒！！！


## 印出 dictionary

In [ ]:
for p_id, p_info in prsn.items():
  print(p_id)
  for key in p_info:
    print(key + ' : ' , p_info[key])

person 1
title_key :  渾然天成的美景
comment_key :  相較於華麗浮誇的裝飾，涵碧樓的住宿體驗像回到家的自在，木質調的房間寬敞舒適，打開窗外將日月潭美景近收眼底，令人回味無窮。
rating_key :  5 STARS
person 2
title_key :  超級推薦的頂級飯店！
comment_key :  頂級的待客服務與美景，住過就永難忘卻那邊的美好，超五星頂級飯店當之無愧。 有著無敵的湖景與以客為尊的服務著名，游泳池也是標準的50米，在這邊可以完全的放鬆，遠離塵囂，好好享受悠閒的度假時光。
rating_key :  5 STARS
person 3
title_key :  Villa的泳池水質很棒
comment_key :  入住Villa房型總是會對私人泳池有些期待，之前有遇過泳池太小或者泳池水質不夠清潔，讓人不敢下水游泳。淪為泡腳池。然而涵碧樓Villa的泳池相當寬敞，而且水質清澈，讓人相當舒適放鬆。非常棒！
rating_key :  5 STARS
person 4
title_key :  氛圍服務硬體都棒，父母孩子三代六人享受了一次難以忘懷的旅遊。
comment_key :  歷史位置創新建築，獨一無二無可取代。服務人員誠懇有經驗，三天兩夜都在飯店裡游走，感受禪意，簡約寧靜。不僅一生要來一次，每年都要來。涵碧樓常駐我心。沐浴用品可再升級。
rating_key :  5 STARS
person 5
title_key :  期待值太高⋯⋯⋯⋯⋯
comment_key :  住過太多飯店，對涵碧樓是相對有期待的，但入住過後，就覺得非常普通，躺在床上，沒想到肩膀上一陣刺痛，以爲漏電，沒想到又在次感到刺痛，尋找發現枕頭上一隻大紅蟻。早餐服務人員素質也有待加強，入座後詢問要咖啡或茶，等了很久還是沒來，像是忘了⋯⋯⋯⋯，有住過就好，沒什麼重複入住的興趣。
rating_key :  2 STARS
person 6
title_key :  值得一去的飯店
comment_key :  整體服務品質都很好，房間很大，提供嬰兒床、浴盆、消毒鍋。疫情期間早餐比較凌亂點其他還不錯。日月潭最美的景色從陽台就可以看得到了。
rating_key :  5 STARS
person 7
title_key :  美好

## 印出 json

In [ ]:
print(jsdic)

{
    "person 1": {
        "comment_key": "相較於華麗浮誇的裝飾，涵碧樓的住宿體驗像回到家的自在，木質調的房間寬敞舒適，打開窗外將日月潭美景近收眼底，令人回味無窮。",
        "rating_key": "5 STARS",
        "title_key": "渾然天成的美景"
    },
    "person 10": {
        "comment_key": "涵碧樓的服務人員相對雲品來說比較專業，從停車至大廳門口開始，都有人帶領，check in後至大廳等待，有好喝的梅子紅茶，還有味道非常不錯的溫熱毛巾，之後也有專人帶領介紹環境和客房，服務態度有五星級飯店的水準。客房古樸，湖景極佳，很適合情侶夫妻來放鬆約會。沒有親子設施，所以帶小孩來我還是會選擇雲品。藝術品商店的店員服務業很好，且對商品的解說詳盡，給予高度好評。",
        "rating_key": "5 STARS",
        "title_key": "適合約會，不宜親子"
    },
    "person 11": {
        "comment_key": "跨年選擇來這裡過，整體上很靜悠，很古典的氛圍，房間是木質家具跟許多花的擺飾。飯店設備最令我印象深刻的是湯池，可以白天跟晚上都來泡一下，景色不錯。房間硬體方面，其實比較中規中矩，感覺沒有更新，電視跟音響設備是舊式,似乎不是很好操控跟使用。整體飯店屬於比較昏暗，如果喜歡明亮的旅客，可能會不習慣。",
        "rating_key": "4 STARS",
        "title_key": "古典飯店"
    },
    "person 12": {
        "comment_key": "氣氛沈靜、環境優美，觀景平台面對日月潭湖景，非常壯觀，服務人員態度良好，部分設施比較舊式，可以再加強！",
        "rating_key": "4 STARS",
        "title_key": "環境優美"
    },
    "person 13": {
        "comment_key": "久聞涵碧樓的高質感住宿環境，終於有幸一窺它的真面目。入口大廳處其實是位於飯店的八樓，從八樓俯視日月潭美景，真的

## 印出 DataFrame

In [ ]:
# 印出dataframe
df = pd.DataFrame(dic)  
print(df)

                          title_keys  ... comment_time
0                            值得一去的飯店  ...     2021年11月
1                            美好回憶的旅行  ...      2021年3月
2                           春節假期前涵碧樓  ...      2021年2月
3               極簡、禪風、Ongoing Style。  ...      2021年2月
4                          適合約會，不宜親子  ...      2021年1月
5                        曾經風光的雅致渡假飯店  ...      2021年1月
6                 風景無敵東方餐廳菜色普通用餐環境壅擠  ...     2020年11月
7                      最美的風景是房間內的蜘蛛絲  ...      2020年9月
8   訂出高房價，就應該提供住客高品質的住宿經驗！涵碧樓該往前走了 !  ...      2020年6月
9                  旅館管理需要加強，旅客隱私沒有保障  ...      2020年4月
10                              古典飯店  ...      2020年1月
11                              環境優美  ...     2019年12月
12                          高質感的住宿環境  ...     2019年11月
13                             美好的地方  ...      2019年8月
14   地理位置和服務極佳的飯店、有很棒的三溫暖澡堂、飲食水平差強人意  ...      2019年8月
15                             悠閒的地方  ...      2019年4月
16                            最喜歡的飯店  ...      2019年4月
17        

# SQL


以下區段是SQL語法

## Import and create path

In [ ]:
import sqlite3
from google.colab import drive
import time
import datetime
drive.mount('/content/drive')
myPath = '/content/drive/MyDrive/Colab Notebooks/sqlite_db/'
conn = sqlite3.connect(myPath + 'myDatabase.db')
c = conn.cursor()
conn.commit()



Mounted at /content/drive


## Delete Table 

In [ ]:
# DELETE TABLE 
c.execute("DROP TABLE review_infos")
print("Drop table successfully")
# conn.close()

Drop table successfully


## Create and Insert Table

In [ ]:
conn.execute('''CREATE TABLE review_infos(p TEXT PRIMARY KEY NOT NULL, 
                      title TEXT NOT NULL,
                      comment TEXT NOT NULL, 
                      rating TEXT NOT NULL,
                      comment_time TEXT );''')
print('table create succesfully')
# for title in titles_new:
conn.execute("INSERT INTO review_infos (p,title,comment,rating) VALUES ('1', 'Paul', '32', 'California')")
print("insert successfully")


conn.commit()


table create succesfully
insert successfully


## Select Table(Print Table) 

In [ ]:
cursor = c.execute("SELECT p, title, comment, rating  from review_infos")
for row in cursor:
  print('person: ' + row[0])
  print('title: ' + row[1])
  print('comment:'  + row[2])
  print('rating: ' + row[3])
print("Operation done successfully") 
# conn.close()

person: 1
title: Paul
comment:32
rating: California
Operation done successfully


## Delete Table of dataframe

In [ ]:
# DELETE TABLE 
c.execute("DROP TABLE review_infos_df")
print("Drop table successfully")
# conn.close()

Drop table successfully


## Use Dataframe to create table

In [ ]:

conn.execute('''CREATE TABLE review_infos_df(
                      ID INTEGER PRIMARY KEY AUTOINCREMENT,
                      title_keys TEXT NOT NULL,
                      comment_keys TEXT NOT NULL, 
                      rating_keys TEXT NOT NULL,
                      comment_time TEXT NOT NULL,
                      insert_time DATETIME );''')
print('table create succesfully')
df.to_sql('review_infos_df', conn, if_exists='append' , index=False)

table create succesfully


In [ ]:

for i in range(1, count+1):
  time_range = datetime.timedelta(hours = 8)
  local_time = datetime.datetime.now()
  new_time = local_time + time_range
  # timeString = time.strftime("%Y/%m/%d %H:%M:%S", local_time) 
  conn.execute("UPDATE review_infos_df set insert_time = ? where ID= ?",[new_time,str(i)])
conn.commit()
print ("Total number of rows updated :", conn.total_changes)

Total number of rows updated : 40


## Testing

In [ ]:
for i in range(1 , count):
  print(i)

1
2
3
4


In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [ ]:
df.to_sql('review_info_tt', con=engine)
engine.execute("SELECT * FROM review_info_test").fetchall()

[(0, '值得一去的飯店', '整體服務品質都很好，房間很大，提供嬰兒床、浴盆、消毒鍋。疫情期間早餐比較凌亂點其他還不錯。日月潭最美的景色從陽台就可以看得到了。', '5 STARS', '2021年11月'),
 (1, '美好回憶的旅行', '每年都會來涵碧樓渡假，從下車到進大廳，服務人員都對我們親切的問候，感覺很溫馨。這次的旅行是為了慶祝週年紀念，謝謝涵碧樓準備的花盒，很美.讓我們很開心感動..很喜歡這裏的氣氛，任何一個角落都很美.飯店質感和服務很好，風景很優美..會讓人一來再來..', '5 STARS', '2021年3月'),
 (2, '春節假期前涵碧樓', '春節假期前，突然訂到涵碧樓！就提早由台北回來台中，隔天到日月潭18000屬於2-3樓，位置特別拜託才給我們泳池正上面三樓，View挺好可惜空污浪費這好山好水。服務接待有超五星等級。建築物是有點老舊設施也看出歲月！但不影響心情，房內空間寬敞感受寧靜，思考的好地方！', '5 STARS', '2021年2月'),
 (3, '極簡、禪風、Ongoing Style。', '四時皆美的日月潭，有如移光借影的明鏡，現在的涵碧樓很不一樣! 喜歡漫步泳池畔，大廳兩旁，一樓的瀑布竹林水池和通往涵碧步道的泳池旁小徑到涵碧步道，很棒的一次家庭旅行。一家三人開了二個房(連通房)的確住的舒適，但兒子的房間落塵灰塵異常的多，鼻子過敏不舒服，小遺憾。', '5 STARS', '2021年2月'),
 (4, '適合約會，不宜親子', '涵碧樓的服務人員相對雲品來說比較專業，從停車至大廳門口開始，都有人帶領，check in後至大廳等待，有好喝的梅子紅茶，還有味道非常不錯的溫熱毛巾，之後也有專人帶領介紹環境和客房，服務態度有五星級飯店的水準。客房古樸，湖景極佳，很適合情侶夫妻來放鬆約會。沒有親子設施，所以帶小孩來我還是會選擇雲品。藝術品商店的店員服務業很好，且對商品的解說詳盡，給予高度好評。', '5 STARS', '2021年1月')]

In [ ]:
c.execute("DROP TABLE review_info_test")
print("Drop table successfully")

OperationalError: ignored

## Select Table(Print Table) Of DataFrame

In [ ]:
cursor = c.execute("SELECT  ID, title_keys, comment_keys, rating_keys, comment_time, insert_time  from review_infos_df")
for row in cursor:
  print('ID: '+ str(row[0]))
  print('title: ' + row[1])
  print('comment:' + row[2])
  print('rating: ' + row[3])
  print('comment_time: ' + row[4])
  print('insert_time: ' + row[5])
  print('\n')
print("Operation done successfully") 
# conn.close()

ID: 1
title: 值得一去的飯店
comment:整體服務品質都很好，房間很大，提供嬰兒床、浴盆、消毒鍋。疫情期間早餐比較凌亂點其他還不錯。日月潭最美的景色從陽台就可以看得到了。
rating: 5 STARS
comment_time: 2021年11月
insert_time: 2021-12-27 16:55:53.646652


ID: 2
title: 美好回憶的旅行
comment:每年都會來涵碧樓渡假，從下車到進大廳，服務人員都對我們親切的問候，感覺很溫馨。這次的旅行是為了慶祝週年紀念，謝謝涵碧樓準備的花盒，很美.讓我們很開心感動..很喜歡這裏的氣氛，任何一個角落都很美.飯店質感和服務很好，風景很優美..會讓人一來再來..
rating: 5 STARS
comment_time: 2021年3月
insert_time: 2021-12-27 16:55:53.652419


ID: 3
title: 春節假期前涵碧樓
comment:春節假期前，突然訂到涵碧樓！就提早由台北回來台中，隔天到日月潭18000屬於2-3樓，位置特別拜託才給我們泳池正上面三樓，View挺好可惜空污浪費這好山好水。服務接待有超五星等級。建築物是有點老舊設施也看出歲月！但不影響心情，房內空間寬敞感受寧靜，思考的好地方！
rating: 5 STARS
comment_time: 2021年2月
insert_time: 2021-12-27 16:55:53.652476


ID: 4
title: 極簡、禪風、Ongoing Style。
comment:四時皆美的日月潭，有如移光借影的明鏡，現在的涵碧樓很不一樣! 喜歡漫步泳池畔，大廳兩旁，一樓的瀑布竹林水池和通往涵碧步道的泳池旁小徑到涵碧步道，很棒的一次家庭旅行。一家三人開了二個房(連通房)的確住的舒適，但兒子的房間落塵灰塵異常的多，鼻子過敏不舒服，小遺憾。
rating: 5 STARS
comment_time: 2021年2月
insert_time: 2021-12-27 16:55:53.652489


ID: 5
title: 適合約會，不宜親子
comment:涵碧樓的服務人員相對雲品來說比較專業，從停車至大廳門口開始，都有人帶領，check in後至大廳等待，有好喝的梅子紅茶，還有味道非常